# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor

# Data preparation

Read data

In [2]:
data = pd.read_excel('data.xlsx')

Sorting by year (avoids misshifting)

In [3]:
data = data.sort_values(by='Year')

Define target columns for model and add new column for previous year pending cases

In [4]:
target_specific = [column for column in data.columns if 'CC' in column and column != 'CC_all']
target_all = 'CC_all'
for pc_column in [col for col in data.columns if 'PC' in col]:
    data[pc_column + '_prev_year'] = data.groupby(['Court', 'Municipality', 'Bench'])[pc_column].shift(1)


Drop Incoming Cases and Pending Cases columns (unknown when predicting)

In [5]:
data = data.drop(columns=[col for col in data.columns if 'PC_all' in col or 'IC' in col or ('PC' in col and '_prev_year' not in col)])

Encode categorical columns

In [6]:
categorical_columns = ['Court', 'Municipality', 'Bench']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

Drop 'Informatic People' variable - only 1 entry in 2017, with value 1

In [7]:
data = data.drop(columns=['Informatic People'])

Drop Justice Officials - sum of 6 prior columns

In [8]:
data = data.drop(columns=['Justice Officials'])

Train test split

In [9]:
data = data.drop(columns=['Year'])

In [10]:
data = data.dropna(axis=0, how='any')


train_X, test_X, train_y, test_y = train_test_split(data.drop(columns=target_specific+[target_all]), data[target_specific], test_size=0.2, random_state=42)

In [11]:
train_X_lr = train_X.sample(frac=0.33, random_state=42)
train_y_lr = train_y.loc[train_X_lr.index]

train_X_rf = train_X.drop(train_X_lr.index).sample(frac=0.5, random_state=42)
train_y_rf = train_y.loc[train_X_rf.index]

train_X_xgb = train_X.drop(train_X_lr.index).drop(train_X_rf.index)
train_y_xgb = train_y.loc[train_X_xgb.index]

In [15]:
test_y = test.loc[test['Year'].isin([2021,2022]), target_specific]
test_X = test.loc[test['Year'].isin([2021,2022])].drop(columns=target_specific + ['Year'] + [target_all])

train_y_lr = train.loc[train['Year'].isin([2016, 2021]), target_specific]
train_X_lr = train.loc[train['Year'].isin([2016, 2021])].drop(columns=target_specific + ['Year'] + [target_all])

train_y_rf = train.loc[train['Year'].isin([2017, 2020]), target_specific]
train_X_rf = train.loc[train['Year'].isin([2017, 2020])].drop(columns=target_specific + ['Year'] + [target_all])

train_y_xgb = train.loc[train['Year'].isin([2018, 2019]), target_specific]
train_X_xgb = train.loc[train['Year'].isin([2018, 2019])].drop(columns=target_specific + ['Year'] + [target_all])


Features

In [11]:
print('Court')
print('Municipality')
print('Bench')
for col in train_X.columns:
    print(col) if 'Court' not in col and 'Municipality' not in col and 'Bench' not in col else None

Court
Municipality
Bench
Judges
Justice Secretary
Law Clerck
Auxiliar Clerck
Administrative/Technical People
Operational/Auxiliar People
PC_Civil_prev_year
PC_Criminal_prev_year
PC_labor_prev_year
PC_criminal_labor_prev_year
PC_tutelar_prev_year
PC_militar_prev_year


# First models

Linear Regression

In [12]:
lr = LinearRegression()

lr.fit(train_X, train_y)

predictions_lr = lr.predict(test_X)
predictions_lr = predictions_lr.round(0)


mse_lr = mean_squared_error(test_y, predictions_lr)
mae_lr = mean_absolute_error(test_y, predictions_lr)
r2_lr = r2_score(test_y, predictions_lr)

print(f"Mean Squared Error: {mse_lr}, Mean Absolute Error: {mae_lr}, R2 Score: {r2_lr}")

Mean Squared Error: 97540.49385853451, Mean Absolute Error: 80.13553578991953, R2 Score: 0.9029555676722651


Decision Tree

In [ ]:
dt = DecisionTreeRegressor(random_state=42)

dt.fit(train_X, train_y)

predictions_dt = dt.predict(test_X)
predictions_dt = predictions_dt.round(0)

mse_dt = mean_squared_error(test_y, predictions_dt)
mae_dt = mean_absolute_error(test_y, predictions_dt)
r2_dt = r2_score(test_y, predictions_dt)

print(f"Mean Squared Error: {mse_dt}, Mean Absolute Error: {mae_dt}, R2 Score: {r2_dt}")

Mean Squared Error: 133589.08831003812, Mean Absolute Error: 47.48814061838204, R2 Score: 0.8604446716860022


In [28]:
print(dt.max_features_)
print(dt.n_features_in_)
print(dt.n_outputs_)
dt.tree_.max_depth

239
239
6


36

XGBoost

In [20]:
xgb = XGBRegressor(random_state=42)

xgb.fit(train_X, train_y)

predictions_xgb = xgb.predict(test_X)
predictions_xgb = predictions_xgb.round(0)

mse_xgb = mean_squared_error(test_y, predictions_xgb)
mae_xgb = mean_absolute_error(test_y, predictions_xgb)
r2_xgb = r2_score(test_y, predictions_xgb)

print(f"Mean Squared Error: {mse_xgb}, Mean Absolute Error: {mae_xgb}, R2 Score: {r2_xgb}")

Mean Squared Error: 103715.0, Mean Absolute Error: 37.90321731567383, R2 Score: 0.9178037047386169


Random Forest

In [21]:
rf = RandomForestRegressor(random_state=42)

rf.fit(train_X, train_y)

predictions_rf = rf.predict(test_X)
predictions_rf = predictions_rf.round(0)

mse_rf = mean_squared_error(test_y, predictions_rf)
mae_rf = mean_absolute_error(test_y, predictions_rf)
r2_rf = r2_score(test_y, predictions_rf)

print(f"Mean Squared Error: {mse_rf}, Mean Absolute Error: {mae_rf}, R2 Score: {r2_rf}")

Mean Squared Error: 97812.69822109275, Mean Absolute Error: 38.352181279119016, R2 Score: 0.9067521314632149


In [24]:
print(rf.n_features_in_)
print(rf.n_outputs_)
len(rf.estimators_)

239
6


100

In [26]:
maxd= 0
for t in rf.estimators_:
    if t.tree_.max_depth > maxd:
        maxd = t.tree_.max_depth
print(maxd)

41


Multi Layer Perceptron

In [17]:
mlp = MLPRegressor(random_state=42, max_iter=1000)

mlp.fit(train_X, train_y)

predictions_mlp = mlp.predict(test_X)
predictions_mlp = predictions_mlp.round(0)

mse_mlp = mean_squared_error(test_y, predictions_mlp)
mae_mlp = mean_absolute_error(test_y, predictions_mlp)
r2_mlp = r2_score(test_y, predictions_mlp)

print(f"Mean Squared Error: {mse_mlp}, Mean Absolute Error: {mae_mlp}, R2 Score: {r2_mlp}")

Mean Squared Error: 107097.0897924608, Mean Absolute Error: 58.303261329944945, R2 Score: -6.502883494811679


Check correlations between the errors of each model

In [17]:
residual_lr = test_y.values - predictions_lr
residual_xgb = test_y.values - predictions_xgb
residual_rf = test_y.values - predictions_rf

In [18]:
correlation_results = {}

for i, target in enumerate(target_specific):
    # Build a DataFrame for the residuals for this target
    df_target = pd.DataFrame({
        'Linear Regression': residual_lr[:, i],
        'XGBoost': residual_xgb[:, i],
        'Random Forest': residual_rf[:, i]
    })

    correlation_results[target] = df_target.corr()

for target, corr_matrix in correlation_results.items():
    print(f"Correlation matrix for {target}:")
    print(corr_matrix)
    print("\n")


model_pairs = [
    ('Linear Regression', 'XGBoost'),
    ('Linear Regression', 'Random Forest'),
    ('XGBoost', 'Random Forest')
]

avg_correlations = {pair: [] for pair in model_pairs}

for i, target in enumerate(target_specific):
    df_target = pd.DataFrame({
        'Linear Regression': residual_lr[:, i],
        'XGBoost': residual_xgb[:, i],
        'Random Forest': residual_rf[:, i]
    })
    corr = df_target.corr()
    for pair in model_pairs:
        avg_correlations[pair].append(corr.loc[pair[0], pair[1]])


print("Average Correlations Across Targets:")
for pair, values in avg_correlations.items():
    avg_corr = np.mean(values)
    print(f"{pair}: {avg_corr:.3f}")

Correlation matrix for CC_Civil:
                   Linear Regression   XGBoost  Random Forest
Linear Regression           1.000000  0.418172       0.520765
XGBoost                     0.418172  1.000000       0.898200
Random Forest               0.520765  0.898200       1.000000


Correlation matrix for CC_Criminal:
                   Linear Regression   XGBoost  Random Forest
Linear Regression           1.000000  0.075718       0.489982
XGBoost                     0.075718  1.000000       0.460033
Random Forest               0.489982  0.460033       1.000000


Correlation matrix for CC_labor:
                   Linear Regression   XGBoost  Random Forest
Linear Regression           1.000000  0.753833       0.827440
XGBoost                     0.753833  1.000000       0.870847
Random Forest               0.827440  0.870847       1.000000


Correlation matrix for CC_criminal_labor:
                   Linear Regression   XGBoost  Random Forest
Linear Regression           1.000000  0.6824

Best models: XGBoost and Random Forests

# Hyper parameter tuning

Random Forest

In [19]:
# Define the parameter grid
param_grid = {'n_estimators': range(10,201,10), 'max_depth': range(1, 21)}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Perform Grid Search with Cross-Validation
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(train_X, train_y)

# Get the best number of estimators
best_rf = grid_search_rf.best_estimator_
best_n_estimators = grid_search_rf.best_params_['n_estimators']
best_max_depth = grid_search_rf.best_params_['max_depth']
print(f"Best number of estimators: {best_n_estimators}")
print(f"Best max depth: {best_max_depth}")

Best number of estimators: 150
Best max depth: 17


In [21]:
best_rf.predict(test_X)

mse_brf = mean_squared_error(test_y, best_rf.predict(test_X))
mae_brf = mean_absolute_error(test_y, best_rf.predict(test_X))
r2_brf = r2_score(test_y, best_rf.predict(test_X))

print(f"Mean Squared Error: {mse_brf}, Mean Absolute Error: {mae_brf}, R2 Score: {r2_brf}")

Mean Squared Error: 101820.5675597646, Mean Absolute Error: 39.17148055688731, R2 Score: 0.9073725601697918


XGBoost

In [22]:
param_grid_xgb = {
    'n_estimators': range(10, 101, 5),
    'max_depth': [3, 5, 7],
}

xgb = XGBRegressor(random_state=42)

grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_xgb.fit(train_X, train_y)

best_xgb = grid_search_xgb.best_estimator_
grid_search_xgb.best_params_

{'max_depth': 5, 'n_estimators': 10}

In [23]:
best_xgb.predict(test_X)

mse_bxgb = mean_squared_error(test_y, best_xgb.predict(test_X))
mae_bxgb = mean_absolute_error(test_y, best_xgb.predict(test_X))
r2_bxgb = r2_score(test_y, best_xgb.predict(test_X))

print(f"Mean Squared Error: {mse_bxgb}, Mean Absolute Error: {mae_bxgb}, R2 Score: {r2_bxgb}")

Mean Squared Error: 107589.7109375, Mean Absolute Error: 46.88821029663086, R2 Score: -0.8458577990531921


Same takeaway

# Ensemble

Stacking

In [194]:
from sklearn.ensemble import StackingRegressor
from sklearn.multioutput import MultiOutputRegressor

seed = 42

lr = LinearRegression()
dt = DecisionTreeRegressor()#random_state=seed)
xgb = XGBRegressor()#random_state=seed)
rf = RandomForestRegressor()#random_state=seed)

estimators = [
    ('lr', lr),
    ('dt', dt),
    ('xgb', xgb),
    ('rf', rf)
]

meta_model = LinearRegression()

stacking_regressor = StackingRegressor(estimators=estimators,
                                       final_estimator=meta_model,
                                       cv=5,
                                       n_jobs=-1)

multi_output_regressor = MultiOutputRegressor(stacking_regressor)

multi_output_regressor.fit(data_train_X, data_train_y)

predictions_stacking = multi_output_regressor.predict(data_test_X)
predictions_stacking = predictions_stacking.round(0)

mse_stacking = mean_squared_error(data_test_y, predictions_stacking)
mae_stacking = mean_absolute_error(data_test_y, predictions_stacking)
r2_stacking = r2_score(data_test_y, predictions_stacking)

print(f"Mean Squared Error: {mse_stacking}, Mean Absolute Error: {mae_stacking}, R2 Score: {r2_stacking}")

KeyboardInterrupt: 

Weighted Average Ensemble

In [22]:
print('Linear Regression:')
lre = LinearRegression()
lre.fit(train_X_lr, train_y_lr)
predictions_lre = lre.predict(test_X)
predictions_lre = predictions_lre.round(0)
mse_lre = mean_squared_error(test_y, predictions_lre)
mae_lre = mean_absolute_error(test_y, predictions_lre)
r2_lre = r2_score(test_y, predictions_lre)
print(f"Mean Squared Error: {mse_lre}, Mean Absolute Error: {mae_lre}, R2 Score: {r2_lre}")

print('Random Forest:')
rfe = RandomForestRegressor(random_state=42)
rfe.fit(train_X_rf, train_y_rf)
predictions_rfe = rfe.predict(test_X)
predictions_rfe = predictions_rfe.round(0)
mse_rfe = mean_squared_error(test_y, predictions_rfe)
mae_rfe = mean_absolute_error(test_y, predictions_rfe)
r2_rfe = r2_score(test_y, predictions_rfe)
print(f"Mean Squared Error: {mse_rfe}, Mean Absolute Error: {mae_rfe}, R2 Score: {r2_rfe}")

print('XGBoost:')
xgbe = XGBRegressor(random_state=42)
xgbe.fit(train_X_xgb, train_y_xgb)
predictions_xgbe = xgbe.predict(test_X)
predictions_xgbe = predictions_xgbe.round(0)
mse_xgbe = mean_squared_error(test_y, predictions_xgbe)
mae_xgbe = mean_absolute_error(test_y, predictions_xgbe)
r2_xgbe = r2_score(test_y, predictions_xgbe)
print(f"Mean Squared Error: {mse_xgbe}, Mean Absolute Error: {mae_xgbe}, R2 Score: {r2_xgbe}")

Linear Regression:
Mean Squared Error: 119428.05506141466, Mean Absolute Error: 96.8818297331639, R2 Score: 0.9025478952012914
Random Forest:
Mean Squared Error: 118831.54150783566, Mean Absolute Error: 45.345404489623036, R2 Score: 0.8290675274836977
XGBoost:
Mean Squared Error: 156936.90625, Mean Absolute Error: 49.464420318603516, R2 Score: 0.7158792614936829


In [23]:
import numpy as np
from scipy.optimize import minimize

n_models = 3  # Number of models
n_targets = 6  # Number of targets
n_samples = test_y.shape[0]  # Number of samples in the test set

predictions = np.array([predictions_lre, predictions_xgbe, predictions_rfe])
y_val = test_y.values

# Assume y_val and predictions are defined as before.
optimal_weights_per_target = np.zeros((n_models, n_targets)) # 4 models, 6 targets

def ensemble_mse_target(weights, predictions_target, y_true_target):
    """
    Compute MSE for a single target variable.
    """
    # Weighted prediction for one target: dot product of weights and model predictions.
    ensemble_pred = np.dot(weights, predictions_target)  # predictions_target shape: (n_models, n_samples)
    mse = np.mean((y_true_target - ensemble_pred) ** 2)
    return mse

for i in range(n_targets):  # Assuming 6 targets
    # Extract predictions for target i: shape becomes (n_models, n_samples)
    predictions_i = predictions[:, :, i]
    y_true_i = y_val[:, i]
    
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1)] * n_models
    initial_weights = np.full(n_models, 1.0 / n_models)
    
    result = minimize(ensemble_mse_target, initial_weights, args=(predictions_i, y_true_i),
                      bounds=bounds, constraints=constraints)
    
    optimal_weights_per_target[:, i] = result.x

model_names = ['Linear Regression','XGBoost', 'Random Forest']
print("Optimal Weights and Corresponding Models per Target:")
for i, target in enumerate(target_specific):
    print(f"{target}:")
    for weight, model in zip(optimal_weights_per_target[:, i], model_names):
        print(f"  {model}: {weight:.4f}")

# Compute ensemble predictions for each target using the corresponding weights
ensemble_pred_separate = np.zeros((n_samples, n_targets))
for i in range(n_targets):
    ensemble_pred_separate[:, i] = np.dot(optimal_weights_per_target[:, i], predictions[:, :, i])


Optimal Weights and Corresponding Models per Target:
CC_Civil:
  Linear Regression: 0.3333
  XGBoost: 0.3333
  Random Forest: 0.3333
CC_Criminal:
  Linear Regression: 0.6031
  XGBoost: 0.2752
  Random Forest: 0.1217
CC_labor:
  Linear Regression: 0.0984
  XGBoost: 0.3866
  Random Forest: 0.5149
CC_criminal_labor:
  Linear Regression: 0.7945
  XGBoost: 0.2055
  Random Forest: 0.0000
CC_tutelar:
  Linear Regression: 0.2088
  XGBoost: 0.2527
  Random Forest: 0.5385
CC_militar:
  Linear Regression: 0.5326
  XGBoost: 0.0000
  Random Forest: 0.4674


In [24]:
ensemble_mse = mean_squared_error(y_val, ensemble_pred_separate)
ensemble_mae = mean_absolute_error(y_val, ensemble_pred_separate)
ensemble_r2 = r2_score(y_val, ensemble_pred_separate)

print(f"Mean Squared Error: {ensemble_mse}, Mean Absolute Error: {ensemble_mae}, R2 Score: {ensemble_r2}")

Mean Squared Error: 83175.4603505669, Mean Absolute Error: 53.903178006522865, R2 Score: 0.9405220533711846


In [34]:
for _ in range(10):

    rfr = RandomForestRegressor()
    rfr.fit(train_X, train_y)
    predictions_rfr = rfr.predict(test_X)
    predictions_rfr = predictions_rfr.round(0)

    mse_rfr = mean_squared_error(test_y, predictions_rfr)
    mae_rfr = mean_absolute_error(test_y, predictions_rfr)
    r2_rfr = r2_score(test_y, predictions_rfr)
    print(f"Mean Squared Error: {mse_rfr}, Mean Absolute Error: {mae_rfr}, R2 Score: {r2_rfr}")

Mean Squared Error: 116259.46293943243, Mean Absolute Error: 39.79500211774671, R2 Score: 0.9051820007427479
Mean Squared Error: 103286.13002964847, Mean Absolute Error: 38.64464210080474, R2 Score: 0.9106146356425548
Mean Squared Error: 112395.8958068615, Mean Absolute Error: 39.664548919949176, R2 Score: 0.9097691849497639
Mean Squared Error: 114282.16539601861, Mean Absolute Error: 39.40724269377383, R2 Score: 0.9069180521948127
Mean Squared Error: 105342.02117746715, Mean Absolute Error: 38.56077933079204, R2 Score: 0.9101732744556772
Mean Squared Error: 106317.28759000421, Mean Absolute Error: 39.302414231257934, R2 Score: 0.9036262974177296
Mean Squared Error: 100831.59085133417, Mean Absolute Error: 38.49047013977129, R2 Score: 0.9070415002184289
Mean Squared Error: 104960.95235069886, Mean Absolute Error: 38.71812791190174, R2 Score: 0.9073506505111171
Mean Squared Error: 109341.61562897079, Mean Absolute Error: 38.71685726387125, R2 Score: 0.9065626985094976
Mean Squared Error